## 🎨🧩 Step 1: Public Artworks Data Modeling

### Import Libraries

In [1]:
# Import Libraries
import osmnx as ox # to fetch data from OpenStreetMap
import geopandas as gpd # to work with geospatial data
import pandas as pd
import numpy as np
import re
from geopy.geocoders import Nominatim
from tqdm import tqdm

### Create the tag to pull galleries from tourism section

In [2]:
tags = {
        "tourism": "artwork"
       }

### Fetch Berlin Geometries

In [3]:
public_artworks_raw = ox.features_from_place("Berlin, Germany", tags)


### Display basic info

In [4]:
print(f"Number of artworks entries fetched: {len(public_artworks_raw)}")
public_artworks_raw.head()

Number of artworks entries fetched: 2609


geometry artwork_type material  \
element id                                                           
node    28341970   POINT (13.62509 52.44417)    sculpture    metal   
        243487615  POINT (13.36187 52.51007)       statue      NaN   
        255049659  POINT (13.16499 52.43388)       statue      NaN   
        258485628  POINT (13.47316 52.52873)    sculpture    stone   
        262455591  POINT (13.39274 52.51726)       statue      NaN   

                                                        name  tourism  \
element id                                                              
node    28341970                                Seepferdchen  artwork   
        243487615                             Richard Wagner  artwork   
        255049659                           Flensburger Löwe  artwork   
        258485628                                  Begegnung  artwork   
        262455591  Reiterstandbild Friedrich II. von Preußen  artwork   

                                                               image  \
element id                                                             
node    28341970                                                 NaN   
        243487615        https://photos.app.goo.gl/9VgMjZd9uThAhJAb8   
        255049659                                                NaN   
        258485628  https://fennpfuhl.digital/img/statue/begegnung...   
        262455591  https://commons.wikimedia.org/wiki/File:Berlin...   

                  wheelchair    wikidata  \
element id                                 
node    28341970         NaN         NaN   
        243487615        yes    Q2148898   
        255049659    limited  Q105045191   
        258485628         no  Q110311378   
        262455591        yes     Q881611   

                                                  wikipedia  \
element id                                                    
node    28341970                                        NaN   
        243487615                en:Richard Wagner Monument   
        255049659                                       NaN   
        258485628                                       NaN   
        262455591  de:Reiterstandbild Friedrichs des Großen   

                                         artist_name  ... website:de  \
element id                                            ...              
node    28341970                                 NaN  ...        NaN   
        243487615                                NaN  ...        NaN   
        255049659  Kopie nach Hermann Wilhelm Bissen  ...        NaN   
        258485628                           Jo Doese  ...        NaN   
        262455591             Christian Daniel Rauch  ...        NaN   

                  geoglyph:language handrail step_count tactile_paving  \
element id                                                               
node    28341970                NaN      NaN        NaN            NaN   
        243487615               NaN      NaN        NaN            NaN   
        255049659               NaN      NaN        NaN            NaN   
        258485628               NaN      NaN        NaN            NaN   
        262455591               NaN      NaN        NaN            NaN   

                  tactile_writing:braille:de  \
element id                                     
node    28341970                         NaN   
        243487615                        NaN   
        255049659                        NaN   
        258485628                        NaN   
        262455591                        NaN   

                  tactile_writing:embossed_printed_letters:de building:colour  \
element id                                                                      
node    28341970                                          NaN             NaN   
        243487615                                         NaN             NaN   
        255049659                                         NaN             NaN   
        258

### Save the raw data to a csv file

- Define file paths

In [5]:
raw_csv_path = "../sources/csv_files/public_artwork_raw.csv"
raw_geojson_path = "../sources/geojson_files/public_artwork_raw.geojson"

- Save csv & geojson to correct folders in sources

In [6]:
public_artworks_raw.to_csv(raw_csv_path, index=False )

In [7]:
public_artworks_raw.to_file(raw_geojson_path, driver="GeoJSON")

### Remove the columns with 75% or more missing data

In [8]:
# Step 1: Calculate percentage of missing values per column
missing_percent = public_artworks_raw.isnull().mean() * 100

# Step 2: Identify columns with less than 75% missing
columns_to_keep = missing_percent[missing_percent < 75].index

# Step 3: Create a new filtered DataFrame
public_artworks_filtered = public_artworks_raw[columns_to_keep]

# Check shape and preview
print(public_artworks_filtered.shape)
public_artworks_filtered.head()

(2609, 10)


geometry artwork_type material  \
element id                                                           
node    28341970   POINT (13.62509 52.44417)    sculpture    metal   
        243487615  POINT (13.36187 52.51007)       statue      NaN   
        255049659  POINT (13.16499 52.43388)       statue      NaN   
        258485628  POINT (13.47316 52.52873)    sculpture    stone   
        262455591  POINT (13.39274 52.51726)       statue      NaN   

                                                        name  tourism  \
element id                                                              
node    28341970                                Seepferdchen  artwork   
        243487615                             Richard Wagner  artwork   
        255049659                           Flensburger Löwe  artwork   
        258485628                                  Begegnung  artwork   
        262455591  Reiterstandbild Friedrich II. von Preußen  artwork   

                                                               image  \
element id                                                             
node    28341970                                                 NaN   
        243487615        https://photos.app.goo.gl/9VgMjZd9uThAhJAb8   
        255049659                                                NaN   
        258485628  https://fennpfuhl.digital/img/statue/begegnung...   
        262455591  https://commons.wikimedia.org/wiki/File:Berlin...   

                     wikidata                        artist_name start_date  \
element id                                                                    
node    28341970          NaN                                NaN        NaN   
        243487615    Q2148898                                NaN        NaN   
        255049659  Q105045191  Kopie nach Hermann Wilhelm Bissen       1938   
        258485628  Q110311378                           Jo Doese        NaN   
        262455591     Q881611             Christian Daniel Rauch        NaN   

                                                   wikimedia_commons  
element id                                                            
node    28341970                                                 NaN  
        243487615                                                NaN  
        255049659           Category:Flensburg Lion (Copy in Berlin)  
        258485628                 Category:Begegnung (Joachim Doese)  
        262455591  Category:Reiterstandbild Friedrichs des Großen...

### Check no Geometries missing

In [9]:
print("Missing geometries:", public_artworks_filtered.geometry.isna().sum())

Missing geometries: 0


### Add Latitude & Longitude columns to filtered geodataframe

In [10]:
# Reproject and extract lat/lon
public_artworks_filtered = public_artworks_filtered.to_crs(epsg=4326)
public_artworks_filtered['geometry'] = public_artworks_filtered['geometry'].apply(
    lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point()
)
public_artworks_filtered['latitude'] = public_artworks_filtered.geometry.y
public_artworks_filtered['longitude'] = public_artworks_filtered.geometry.x

### Verify the lat/lon amounts are correct

In [11]:
print("Latitude range:", public_artworks_filtered["latitude"].min(), "to", public_artworks_filtered["latitude"].max())

print("Longitude range:", public_artworks_filtered["longitude"].min(), "to", public_artworks_filtered["longitude"].max())

Latitude range: 52.3975036 to 52.669101
Longitude range: 13.0913224 to 13.7208069


### Reset index and drop redundant element column

In [12]:
public_artworks_filtered = public_artworks_filtered.reset_index()

# set the id to string
public_artworks_filtered["id"] = public_artworks_filtered["id"].astype(str)

# Drop the redundant column "element"
public_artworks_filtered= public_artworks_filtered.drop(columns=["element"],errors='ignore')
# Show final list of columns
print(public_artworks_filtered.columns.tolist())

['id', 'geometry', 'artwork_type', 'material', 'name', 'tourism', 'image', 'wikidata', 'artist_name', 'start_date', 'wikimedia_commons', 'latitude', 'longitude']


In [13]:
public_artworks_filtered.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2609 entries, 0 to 2608
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   id                 2609 non-null   object  
 1   geometry           2609 non-null   geometry
 2   artwork_type       2402 non-null   object  
 3   material           1185 non-null   object  
 4   name               1776 non-null   object  
 5   tourism            2609 non-null   object  
 6   image              715 non-null    object  
 7   wikidata           1173 non-null   object  
 8   artist_name        1466 non-null   object  
 9   start_date         936 non-null    object  
 10  wikimedia_commons  824 non-null    object  
 11  latitude           2609 non-null   float64 
 12  longitude          2609 non-null   float64 
dtypes: float64(2), geometry(1), object(10)
memory usage: 265.1+ KB


---

## 🎨🔄 Step 2: Public Artworks Data Transformation

### Drop unnecessary columns
    - Dont need Berlin and DE as user knows they are looking for data in Berlin, Germany
    - Drop tourism column as all artworks
    - Drop suburb as it is the same as neighborhood which will be added later to make sure nothing is missing

In [14]:
public_artworks_filtered.drop(columns=['addr:city', 'addr:country', 'tourism', 'addr:suburb'], errors='ignore', inplace=True)

### Copy the filtered database for record purposes and then standardise column names

In [15]:
public_artworks_cleaned = public_artworks_filtered.copy()

def clean_column(col):
    col = col.strip().lower().replace('addr:', '')
    col = col.replace(' ', '_')
    col = re.sub(r'[^a-z0-9_]', '', col)
    return col

public_artworks_cleaned.columns = [clean_column(col) for col in public_artworks_cleaned.columns]

In [16]:
public_artworks_cleaned.columns.tolist()

['id',
 'geometry',
 'artwork_type',
 'material',
 'name',
 'image',
 'wikidata',
 'artist_name',
 'start_date',
 'wikimedia_commons',
 'latitude',
 'longitude']

### Fetch Districts & Neighbourhoods

- Load official Berlin districts GeoDataFrame

In [17]:
districts_gdf = gpd.read_file("../sources/geojson_files/lor_ortsteile.geojson")

In [18]:
districts_gdf.head(2)

,gml_id,spatial_name,spatial_alias,spatial_type,OTEIL,BEZIRK,FLAECHE_HA,geometry
0,re_ortsteil.0101,0101,Mitte,Polygon,Mitte,Mitte,1063.8748,"POLYGON ((13.41649 52.52696, 13.41635 52.52702..."
1,re_ortsteil.0102,0102,Moabit,Polygon,Moabit,Mitte,768.7909,"POLYGON ((13.33884 52.51974, 13.33884 52.51974..."


- Reproject GeoDataFrames to EPSG:4326 

In [19]:
public_artworks_cleaned = public_artworks_cleaned.to_crs(epsg=4326)
districts_gdf = districts_gdf.to_crs(epsg=4326)

- Spatial join with district(Bezirk) and Neighbourhood_id(spatial_name)

In [20]:
public_artworks_df_district = gpd.sjoin(
    public_artworks_cleaned,
    districts_gdf[["BEZIRK", "spatial_name","geometry"]],
    how="left",
    predicate="within"
)

- Rename columns for clarity

In [21]:

public_artworks_df_district = public_artworks_df_district.rename(columns={
    "BEZIRK": "district",
    "spatial_name": "neighborhood_id"
}).drop(columns=["index_right"])  

- District mapping (official codes as strings)

In [22]:
district_mapping = {
    'Mitte': '11001001',
    'Friedrichshain-Kreuzberg': '11002002',
    'Pankow': '11003003',
    'Charlottenburg-Wilmersdorf': '11004004',
    'Spandau': '11005005',
    'Steglitz-Zehlendorf': '11006006',
    'Tempelhof-Schöneberg': '11007007',
    'Neukölln': '11008008',
    'Treptow-Köpenick': '11009009',
    'Marzahn-Hellersdorf': '11010010',
    'Lichtenberg': '11011011',
    'Reinickendorf': '11012012'
}

# Apply mapping to create district_id column (string)
public_artworks_df_district['district_id'] = public_artworks_df_district['district'].map(district_mapping).astype(str)

In [23]:
public_artworks_df_district.head()

,id,geometry,artwork_type,material,name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,district,neighborhood_id,district_id
0,28341970,POINT (13.62509 52.44417),sculpture,metal,Seepferdchen,NaN,NaN,NaN,NaN,NaN,52.444170,13.625088,Treptow-Köpenick,0910,11009009
1,243487615,POINT (13.36187 52.51007),statue,NaN,Richard Wagner,https://photos.app.goo.gl/9VgMjZd9uThAhJAb8,Q2148898,NaN,NaN,NaN,52.510066,13.361869,Mitte,0104,11001001
2,255049659,POINT (13.16499 52.43388),statue,NaN,Flensburger Löwe,NaN,Q105045191,Kopie nach Hermann Wilhelm Bissen,1938,Category:Flensburg Lion (Copy in Berlin),52.433884,13.164987,Steglitz-Zehlendorf,0607,11006006
3,258485628,POINT (13.47316 52.52873),sculpture,stone,Begegnung,https://fennpfuhl.digital/img/statue/begegnung...,Q110311378,Jo Doese,NaN,Category:Begegnung (Joachim Doese),52.528735,13.473156,Lichtenberg,1111,11011011
4,262455591,POINT (13.39274 52.51726),statue,NaN,Reiterstandbild Friedrich II. von Preußen,https://commons.wikimedia.org/wiki/File:Berlin...,Q881611,Christian Daniel Rauch,NaN,Category:Reiterstandbild Friedrichs des Großen...,52.517260,13.392744,Mitte,0101,11001001


### Used reverse Geocoding and Nominatim to get Postal code, Street & house number info
    - Created postal_code, street and house_number columns to try get data from the latitude and longitude

In [24]:
tqdm.pandas()

# Initialize geocoder
geolocator = Nominatim(user_agent="gallery_locator")

# Define function to extract postal code and street
def get_postcode_and_street(row):
    try:
        location = geolocator.reverse((row['latitude'], row['longitude']), exactly_one=True)
        address = location.raw.get('address', {})
        postcode = address.get('postcode')
        street = address.get('road') or address.get('pedestrian') or address.get('footway') or address.get('street')
        house_number = address.get('housenumber')
        return pd.Series([postcode, street, house_number])
    except:
        return pd.Series([None, None, None])

# Apply to your DataFrame
public_artworks_df_district[['postal_code', 'street', 'house_number']] = public_artworks_df_district.progress_apply(get_postcode_and_street, axis=1)


100%|██████████| 2609/2609 [1:47:32<00:00,  2.47s/it]  


In [25]:
public_artworks_full = public_artworks_df_district.copy()

### Data Cleaning & Normalization
    - Replace all missing values with Nan

In [26]:
# Replace empty strings with NaN
public_artworks_full.replace('', np.nan, inplace=True)

public_artworks_full.head(2)

,id,geometry,artwork_type,material,name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,district,neighborhood_id,district_id,postal_code,street,house_number
0,28341970,POINT (13.62509 52.44417),sculpture,metal,Seepferdchen,NaN,NaN,NaN,NaN,NaN,52.444170,13.625088,Treptow-Köpenick,0910,11009009,12559,Spreetunnel,None
1,243487615,POINT (13.36187 52.51007),statue,NaN,Richard Wagner,https://photos.app.goo.gl/9VgMjZd9uThAhJAb8,Q2148898,NaN,NaN,NaN,52.510066,13.361869,Mitte,0104,11001001,10785,Tiergartenstraße,None


### Normalize the street name column

In [27]:
def normalize_street_name(name):
    if pd.isna(name):
        return np.nan
    # Replace underscores with spaces
    name = name.replace('_', ' ').replace('-', ' ')
    # Replace 'str.' or 'str' at end with ' Straße'
    name = re.sub(r'\bstr\.?\s*$', ' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'straße' if missing
    name = re.sub(r'(?<!\s)(straße)$', r' Straße', name, flags=re.IGNORECASE)
    # Ensure space before 'allee' if missing
    name = re.sub(r'(?<!\s)(allee)$', r' Allee', name, flags=re.IGNORECASE)
    # Ensure space before 'damm' if missing
    name = re.sub(r'(?<!\s)(damm)$', r' Damm', name, flags=re.IGNORECASE)
    # Ensure space before 'weg' if missing
    name = re.sub(r'(?<!\s)(weg)$', r' Weg', name, flags=re.IGNORECASE)
    # Ensure space before 'graben' if missing
    name = re.sub(r'(?<!\s)(graben)$', r' Graben', name, flags=re.IGNORECASE)
    # Ensure space before 'ufer' if missing
    name = re.sub(r'(?<!\s)(ufer)$', r' Ufer', name, flags=re.IGNORECASE)
    # Ensure space before 'korso' if missing
    name = re.sub(r'(?<!\s)(korso)$', r' Korso', name, flags=re.IGNORECASE)
    # Ensure space before 'zeile' if missing
    name = re.sub(r'(?<!\s)(zeile)$', r' Ziele', name, flags=re.IGNORECASE)
    # Ensure space before 'promenade' if missing
    name = re.sub(r'(?<!\s)(promenade)$', r' Promenade', name, flags=re.IGNORECASE)
    # Ensure space before 'kiez' if missing
    name = re.sub(r'(?<!\s)(kiez)$', r' Kiez', name, flags=re.IGNORECASE)
    # Ensure space before 'platz' if missing
    name = re.sub(r'(?<!\s)(platz)$', r' Platz', name, flags=re.IGNORECASE)
    # Ensure space before 'steig' if missing
    name = re.sub(r'(?<!\s)(steig)$', r' Steig', name, flags=re.IGNORECASE)
    # Remove extra spaces
    name = re.sub(r'\s+', ' ', name).strip()
    return name

In [28]:
public_artworks_full['street'] = public_artworks_full['street'].apply(normalize_street_name)

### Drop unnecessary columns

In [29]:
public_artworks_full.drop(columns=['geometry', 'district'], errors='ignore', inplace=True)

### Convert all text in columns to lowercase to avoid any duplications

In [30]:
text_cols = ["artwork_type", "material", "name", "image", "artist_name", "wikimedia_commons", "street"]

for col in text_cols:
    if col in public_artworks_full.columns:
        public_artworks_full[col] = public_artworks_full[col].apply(
            lambda x: x.strip().lower() if isinstance(x, str) else x
        )

### Check column data types
    - All have correct type allocated so no need to change anything

In [31]:
public_artworks_full.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 2609 entries, 0 to 2608
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2609 non-null   object 
 1   artwork_type       2402 non-null   object 
 2   material           1185 non-null   object 
 3   name               1776 non-null   object 
 4   image              715 non-null    object 
 5   wikidata           1173 non-null   object 
 6   artist_name        1466 non-null   object 
 7   start_date         936 non-null    object 
 8   wikimedia_commons  824 non-null    object 
 9   latitude           2609 non-null   float64
 10  longitude          2609 non-null   float64
 11  neighborhood_id    2609 non-null   object 
 12  district_id        2609 non-null   object 
 13  postal_code        576 non-null    object 
 14  street             576 non-null    object 
 15  house_number       0 non-null      object 
dtypes: float64(2), object

### Remove duplicates
    - In this case we have no duplicates but will keep this check for future runs incase

In [32]:
# See how many duplicate rows exist
public_artworks_full.duplicated().sum()

np.int64(0)

In [33]:
# Display the actual duplicate rows
public_artworks_full[public_artworks_full.duplicated()]

,id,artwork_type,material,name,image,wikidata,artist_name,start_date,wikimedia_commons,latitude,longitude,neighborhood_id,district_id,postal_code,street,house_number


In [34]:
# Remove duplicate rows and reset index
public_artworks_full = public_artworks_full.drop_duplicates().reset_index(drop=True)

### Remove row if name missing

In [35]:
public_artworks_full = public_artworks_full.dropna(subset=['name'])

### Reorder column names to be clearer

In [36]:
public_artworks_listings = public_artworks_full[['id', 'name', 'artwork_type', 'artist_name', 'street', 'neighborhood_id', 
                                                 'district_id', 'postal_code', 'material', 'start_date','image', 'wikidata',  'wikimedia_commons', 'latitude', 'longitude']]

In [37]:
public_artworks_listings.to_csv("../sources/csv_files/public_artwork_listings.csv", index=False)

### Final Summary of cleaned and Transformed Data

In [38]:
# Shape of dataframe
print(f"Number of rows: {public_artworks_listings.shape[0]}")
print(f"Number of columns: {public_artworks_listings.shape[1]}")

Number of rows: 1776
Number of columns: 15


In [39]:
# Column list
print("\nRemaining columns:")
print(public_artworks_listings.columns.tolist())


Remaining columns:
['id', 'name', 'artwork_type', 'artist_name', 'street', 'neighborhood_id', 'district_id', 'postal_code', 'material', 'start_date', 'image', 'wikidata', 'wikimedia_commons', 'latitude', 'longitude']


In [40]:
# Missing values check
missing = public_artworks_listings.isnull().sum()
print("\nMissing values after cleaning and transforming :")
print(missing)


Missing values after cleaning and transforming :
id                      0
name                    0
artwork_type           57
artist_name           429
street               1334
neighborhood_id         0
district_id             0
postal_code          1334
material              782
start_date            923
image                1260
wikidata              699
wikimedia_commons    1080
latitude                0
longitude               0
dtype: int64


In [41]:
# Material Types Frequency
material_counts = public_artworks_listings['material'].value_counts()
material_counts

material
bronze                              369
sandstone                           105
stone                               103
steel                                72
metal                                39
                                   ... 
stainless steel, synthetic resin      1
beton                                 1
sandstone;granite                     1
marble, steel                         1
metal_sheet                           1
Name: count, Length: 126, dtype: int64

In [42]:
# Artwork Types Frequency
artwork_type_counts = public_artworks_listings['artwork_type'].value_counts()
artwork_type_counts

artwork_type
sculpture          858
statue             522
graffiti           102
mural               85
installation        83
relief              25
bust                 8
architecture         7
mosaic               6
stone                5
ensemble             4
stele                4
fountain             2
painting             2
print                1
land_art             1
discs                1
plaque               1
sculpture_brick      1
column               1
Name: count, dtype: int64

In [43]:
# Artwork per district
artworks_per_district = public_artworks_listings['district_id'].value_counts()
artworks_per_district

district_id
11001001    367
11009009    302
11011011    226
11002002    205
11004004    174
11010010    124
11003003    123
11008008     87
11006006     60
11007007     45
11005005     35
11012012     28
Name: count, dtype: int64

In [44]:
# Show first and last date of artworks
public_artworks_listings['start_date'].sort_values()

1150          1647
111           1740
357           1791
382           1793
30      1827..1834
           ...    
2577           NaN
2578           NaN
2582           NaN
2583           NaN
2584           NaN
Name: start_date, Length: 1776, dtype: object

In [45]:
public_artworks_listings.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1776 entries, 0 to 2590
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1776 non-null   object 
 1   name               1776 non-null   object 
 2   artwork_type       1719 non-null   object 
 3   artist_name        1347 non-null   object 
 4   street             442 non-null    object 
 5   neighborhood_id    1776 non-null   object 
 6   district_id        1776 non-null   object 
 7   postal_code        442 non-null    object 
 8   material           994 non-null    object 
 9   start_date         853 non-null    object 
 10  image              516 non-null    object 
 11  wikidata           1077 non-null   object 
 12  wikimedia_commons  696 non-null    object 
 13  latitude           1776 non-null   float64
 14  longitude          1776 non-null   float64
dtypes: float64(2), object(13)
memory usage: 222.0+ KB


In [46]:
public_artworks_listings.head()

,id,name,artwork_type,artist_name,street,neighborhood_id,district_id,postal_code,material,start_date,image,wikidata,wikimedia_commons,latitude,longitude
0,28341970,seepferdchen,sculpture,NaN,spreetunnel,0910,11009009,12559,metal,NaN,NaN,NaN,NaN,52.444170,13.625088
1,243487615,richard wagner,statue,NaN,tiergarten straße,0104,11001001,10785,NaN,NaN,https://photos.app.goo.gl/9vgmjzd9uthahjab8,Q2148898,NaN,52.510066,13.361869
2,255049659,flensburger löwe,statue,kopie nach hermann wilhelm bissen,tiefhorn weg,0607,11006006,14109,NaN,1938,NaN,Q105045191,category:flensburg lion (copy in berlin),52.433884,13.164987
3,258485628,begegnung,sculpture,jo doese,anton saefkow platz,1111,11011011,10369,stone,NaN,https://fennpfuhl.digital/img/statue/begegnung...,Q110311378,category:begegnung (joachim doese),52.528735,13.473156
4,262455591,reiterstandbild friedrich ii. von preußen,statue,christian daniel rauch,unter den linden,0101,11001001,10117,NaN,NaN,https://commons.wikimedia.org/wiki/file:berlin...,Q881611,category:reiterstandbild friedrichs des großen...,52.517260,13.392744


## 🎨🗃️ Step 3: Populate Database

### Import extra Libraries

In [47]:
import psycopg2
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

### Credentials

In [ ]:
user_name=''
password=''

### Create the connection

In [49]:
# Conection
host = ''
port = ''
database = ''
schema=''

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

### Use table tested above with contraints

In [50]:
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.public_artworks (
    id VARCHAR(20) PRIMARY KEY,  
    name VARCHAR(200) NOT NULL,
    artwork_type VARCHAR(200),
    artist_name VARCHAR(200),       
    street VARCHAR(200),
    neighborhood_id VARCHAR(20),
    district_id VARCHAR(200) NOT NULL,
    postal_code VARCHAR(20),
    material VARCHAR(200),
    start_date VARCHAR(50),
    image VARCHAR(250),
    wikidata VARCHAR(250),
    wikimedia_commons VARCHAR(250),             
    latitude DECIMAL(9,6) NOT NULL,
    longitude DECIMAL(9,6) NOT NULL,
    CONSTRAINT district_id_fk FOREIGN KEY (district_id)
        REFERENCES berlin_source_data.districts(district_id)
        ON DELETE RESTRICT
        ON UPDATE CASCADE
);
"""

### Execute the create table query

In [51]:
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction
    print("Table 'public_artworks' created or already exists.")  

Table 'public_artworks' created or already exists.


### Write the table to the layered Database using to.sql()

In [52]:
public_artworks_listings.to_sql(
    'public_artworks',      
    engine,
    schema=schema,
    if_exists='append', # ✅ keeps table, just inserts data
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

DataFrame sent to PostgreSQL using .to_sql() with psycopg2!


### Query the table with SQL

In [53]:
query = f"""
SELECT * 
FROM berlin_source_data.public_artworks
LIMIT 10;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,id,name,artwork_type,artist_name,street,neighborhood_id,district_id,postal_code,material,start_date,image,wikidata,wikimedia_commons,latitude,longitude
0,28341970,seepferdchen,sculpture,None,spreetunnel,0910,11009009,12559,metal,None,None,None,None,52.444170,13.625088
1,243487615,richard wagner,statue,None,tiergarten straße,0104,11001001,10785,None,None,https://photos.app.goo.gl/9vgmjzd9uthahjab8,Q2148898,None,52.510066,13.361869
2,255049659,flensburger löwe,statue,kopie nach hermann wilhelm bissen,tiefhorn weg,0607,11006006,14109,None,1938,None,Q105045191,category:flensburg lion (copy in berlin),52.433884,13.164987
3,258485628,begegnung,sculpture,jo doese,anton saefkow platz,1111,11011011,10369,stone,None,https://fennpfuhl.digital/img/statue/begegnung...,Q110311378,category:begegnung (joachim doese),52.528735,13.473156
4,262455591,reiterstandbild friedrich ii. von preußen,statue,christian daniel rauch,unter den linden,0101,11001001,10117,None,None,https://commons.wikimedia.org/wiki/file:berlin...,Q881611,category:reiterstandbild friedrichs des großen...,52.517260,13.392744
5,276952883,friedrich wilhelm iii. von preußen,statue,friedrich drake,ahorn steig,0104,11001001,10785,concrete,1982,http://commons.wikimedia.org/wiki/file:denkmal...,Q113341316,category:friedrich wilhelm iii by friedrich drake,52.511568,13.364488
6,278707399,büffeljagd,sculpture,fritz schaper,fasanerie allee,0104,11001001,10787,bronze,None,http://commons.wikimedia.org/wiki/file:wiesenj...,Q25063929,category:altgermanische wisentjagd by fritz sc...,52.513325,13.348503
7,278707727,eberjagd,sculpture,karl begas,fasanerie allee,0104,11001001,10787,bronze,None,https://photos.app.goo.gl/vsgmcntm6l29cnyi6,Q24976714,category:eberjagd um 1500 by carl begas,52.513267,13.348622
8,278707884,fuchsjagd,sculpture,wilhelm haverkamp,fasanerie allee,0104,11001001,10787,bronze,None,http://commons.wikimedia.org/wiki/file:fuchsja...,Q24976710,category:churfürstliche fuchsjagd by wilhelm h...,52.511561,13.346312
9,278707964,hasenhetze,sculpture,max baumbach,fasanerie allee,0104,11001001,10787,bronze,None,http://commons.wikimedia.org/wiki/file:hasenhe...,Q24976711,category:hasenhatz der rokokozeit by max baumbach,52.511500,13.346448


### Count unique number of artworks and unique artists

In [54]:
query = f"""
SELECT COUNT(DISTINCT name) AS unique_artwork_names, COUNT(DISTINCT artist_name) AS unique_artist_names
FROM berlin_source_data.public_artworks
LIMIT 10;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,unique_artwork_names,unique_artist_names
0,1546,833


### See which artists have more then 1 artwork

In [55]:
query = f"""
SELECT name, COUNT(*) AS artwork_count
FROM berlin_source_data.public_artworks
GROUP BY name
ORDER BY artwork_count DESC
LIMIT 10
;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,name,artwork_count
0,untitled,14
1,buddy bear,13
2,mutter mit kind,11
3,bär,9
4,sechs liegende elefanten,6
5,sonnenuhr,6
6,buddy bär,5
7,mutter und kind,5
8,löwe,4
9,seelöwe,4


### Count of the different artwork types

In [56]:
query = f"""
SELECT artwork_type, COUNT(*) AS artwork_count
FROM berlin_source_data.public_artworks
GROUP BY artwork_type
ORDER BY artwork_count DESC
;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,artwork_type,artwork_count
0,sculpture,858
1,statue,522
2,graffiti,102
3,mural,85
4,installation,83
5,None,57
6,relief,25
7,bust,8
8,architecture,7
9,mosaic,6


### District with highest artwork count

In [57]:
query = f"""
SELECT di.district_id, di.district, COUNT(*) AS artwork_count
FROM berlin_source_data.public_artworks pa
JOIN berlin_source_data.districts di
ON pa.district_id = di.district_id
GROUP BY di.district_id, di.district
ORDER BY artwork_count DESC
;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,district_id,district,artwork_count
0,11001001,Mitte,367
1,11009009,Treptow-Köpenick,302
2,11011011,Lichtenberg,226
3,11002002,Friedrichshain-Kreuzberg,205
4,11004004,Charlottenburg-Wilmersdorf,174
5,11010010,Marzahn-Hellersdorf,124
6,11003003,Pankow,123
7,11008008,Neukölln,87
8,11006006,Steglitz-Zehlendorf,60
9,11007007,Tempelhof-Schöneberg,45


### Check all districts exist

In [58]:
query = f"""
SELECT pa.district_id, di.district_id, COUNT(*) as artwork_count
FROM berlin_source_data.public_artworks pa
JOIN berlin_source_data.districts di
ON pa.district_id = di.district_id
GROUP BY pa.district_id, di.district_id
ORDER BY pa.district_id;
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df

,district_id,district_id,artwork_count
0,11001001,11001001,367
1,11002002,11002002,205
2,11003003,11003003,123
3,11004004,11004004,174
4,11005005,11005005,35
5,11006006,11006006,60
6,11007007,11007007,45
7,11008008,11008008,87
8,11009009,11009009,302
9,11010010,11010010,124
